# Generating Synthetic scRNAseq Data with CAR Sequence Insertion

This notebook provides a step-by-step process for creating a synthetic single-cell RNA sequencing (scRNAseq) dataset with randomly inserted Chimeric Antigen Receptor (CAR) sequences. This can be useful for testing analysis pipelines or developing new methods without requiring actual sequencing data.

## Overview

We'll create a realistic scRNAseq dataset by:
1. Simulating a gene expression matrix that mimics real T cell populations
2. Generating a synthetic CAR sequence
3. Randomly inserting the CAR sequence into a subset of cells
4. Saving the data in formats compatible with standard scRNAseq analysis tools
5. Visualizing the synthetic data to verify its properties

## 1. Setting Up the Environment

First, let's install and import the necessary packages.

In [ ]:
# Install required packages
!pip install scanpy pandas numpy matplotlib seaborn scikit-learn biopython anndata scipy

In [ ]:
# Import libraries
import os
import random
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import sparse
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from pathlib import Path
from sklearn.mixture import GaussianMixture

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Set plotting defaults
sc.settings.verbosity = 1
sc.settings.set_figure_params(dpi=100, facecolor='white')
%matplotlib inline

## 2. Simulating Gene Expression Data

Next, we'll create a synthetic gene expression matrix that mimics T cell populations. A realistic scRNAseq dataset has:
- Thousands of genes (rows)
- Hundreds to thousands of cells (columns)
- Sparse data (many zeros)
- Cell types with distinct expression patterns
- Technical variation and biological noise

We'll create T cell subpopulations (CD4+, CD8+, etc.) with different expression profiles.

In [ ]:
# Set simulation parameters
n_genes = 5000  # Number of genes to simulate
n_cells = 3000  # Total number of cells
n_cell_types = 5  # Number of cell types (CD4+, CD8+, Tregs, etc.)
sparsity = 0.85  # Fraction of zeros in the dataset

# Create a list of gene names (including some known T cell markers)
t_cell_markers = [
    'CD3D', 'CD3E', 'CD3G',  # T cell markers
    'CD4', 'IL7R',  # CD4+ T cell markers
    'CD8A', 'CD8B',  # CD8+ T cell markers
    'FOXP3', 'IL2RA',  # Regulatory T cell markers
    'CCR7', 'LEF1', 'TCF7',  # Naive T cell markers
    'GZMA', 'GZMB', 'PRF1',  # Effector T cell markers
    'PDCD1', 'HAVCR2', 'LAG3', 'TIGIT'  # Exhausted T cell markers
]

# Generate remaining random gene names
remaining_genes = [f"GENE_{i}" for i in range(n_genes - len(t_cell_markers))]
all_genes = t_cell_markers + remaining_genes

# Define cell type proportions (what fraction of cells are of each type)
cell_type_props = np.array([0.4, 0.3, 0.15, 0.1, 0.05])  # must sum to 1
cell_type_counts = np.round(cell_type_props * n_cells).astype(int)
# Adjust to ensure we have exactly n_cells
cell_type_counts[-1] = n_cells - np.sum(cell_type_counts[:-1])

# Define cell types
cell_types = ['CD4+ T', 'CD8+ T', 'Regulatory T', 'Naive T', 'Effector T']

print(f"Cell type distribution:")
for cell_type, count in zip(cell_types, cell_type_counts):
    print(f"{cell_type}: {count} cells ({count/n_cells*100:.1f}%)")

### 2.1 Create Gene Expression Profiles for Each Cell Type

We'll assign specific expression patterns to each cell type, with higher expression of their characteristic marker genes.

In [ ]:
# Define gene expression parameters for each cell type
gene_params = {}

# For all cell types, set baseline parameters
for gene in all_genes:
    # Mean and dispersion parameters for each gene across all cell types
    # (mean, dispersion) where dispersion controls variance
    gene_params[gene] = [(0.1, 0.5)] * n_cell_types

# Adjust expression of marker genes for specific cell types
# For CD4+ T cells (type 0)
for gene in ['CD3D', 'CD3E', 'CD3G', 'CD4', 'IL7R']:
    gene_params[gene][0] = (2.5, 0.8)  # Higher mean expression, higher variability

# For CD8+ T cells (type 1)
for gene in ['CD3D', 'CD3E', 'CD3G', 'CD8A', 'CD8B']:
    gene_params[gene][1] = (2.5, 0.8)

# For Regulatory T cells (type 2)
for gene in ['CD3D', 'CD3E', 'CD3G', 'CD4', 'FOXP3', 'IL2RA']:
    gene_params[gene][2] = (2.5, 0.8)

# For Naive T cells (type 3)
for gene in ['CD3D', 'CD3E', 'CD3G', 'CCR7', 'LEF1', 'TCF7']:
    gene_params[gene][3] = (2.5, 0.8)

# For Effector T cells (type 4)
for gene in ['CD3D', 'CD3E', 'CD3G', 'GZMA', 'GZMB', 'PRF1']:
    gene_params[gene][4] = (2.5, 0.8)

### 2.2 Generate Expression Matrix Using Negative Binomial Distribution

scRNAseq count data typically follows a negative binomial distribution, which we'll use to generate realistic expression values.

In [ ]:
# Function to generate counts from a negative binomial distribution
def generate_nb_counts(mean, dispersion, size):
    # Convert negative binomial parameters to scipy's parameterization
    var = mean + mean**2 * dispersion
    p = mean / var
    n = mean**2 / (var - mean)
    
    # Generate counts
    counts = np.random.negative_binomial(n, p, size)
    return counts

# Initialize expression matrix
expression_matrix = np.zeros((n_genes, n_cells), dtype=np.float32)

# Generate cell type labels
cell_type_labels = np.zeros(n_cells, dtype=int)
start_idx = 0
for i, count in enumerate(cell_type_counts):
    cell_type_labels[start_idx:start_idx+count] = i
    start_idx += count

# Fill the expression matrix
for i, gene in enumerate(all_genes):
    for cell_type in range(n_cell_types):
        # Get cells of this type
        cell_indices = np.where(cell_type_labels == cell_type)[0]
        n_type_cells = len(cell_indices)
        
        # Get expression parameters for this gene in this cell type
        mean, dispersion = gene_params[gene][cell_type]
        
        # Generate expression values
        counts = generate_nb_counts(mean, dispersion, n_type_cells)
        
        # Add to expression matrix
        expression_matrix[i, cell_indices] = counts

# Add sparsity (replace values with zeros based on the sparsity parameter)
mask = np.random.random(expression_matrix.shape) < sparsity
expression_matrix[mask] = 0

# Convert to sparse matrix (commonly used for scRNAseq data)
expression_matrix_sparse = sparse.csr_matrix(expression_matrix)

print(f"Expression matrix shape: {expression_matrix.shape}")
print(f"Sparsity: {np.sum(expression_matrix == 0) / expression_matrix.size:.2f}")
print(f"Mean expression: {np.mean(expression_matrix):.4f}")

### 2.3 Create AnnData Object

AnnData is a common format for storing scRNAseq data in Python and is used by packages like scanpy.

In [ ]:
# Create AnnData object
adata = sc.AnnData(expression_matrix_sparse.T)  # Note the transpose (cells as rows, genes as columns)

# Add gene names and cell metadata
adata.var_names = all_genes  # Gene names
adata.obs['cell_type'] = [cell_types[label] for label in cell_type_labels]  # Cell type labels
adata.obs['cell_id'] = [f"cell_{i}" for i in range(n_cells)]  # Cell IDs
adata.obs_names = adata.obs['cell_id']

# Add some metadata
adata.uns['cell_types'] = cell_types

# Calculate basic QC metrics
sc.pp.calculate_qc_metrics(adata, inplace=True)

# Display the AnnData object
print(adata)

### 2.4 Visualize the Synthetic Data

Let's check if our synthetic data has the expected properties.

In [ ]:
# Process the data for visualization
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pp.pca(adata, n_comps=50, use_highly_variable=True)
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=20)
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=0.5)

# Plot UMAP colored by cell type
sc.pl.umap(adata, color='cell_type', title='Synthetic T Cell Populations')

# Plot expression of key marker genes
markers_to_plot = [
    'CD4',
    'CD8A',
    'FOXP3',
    'CCR7',
    'GZMA'
]

sc.pl.umap(adata, color=markers_to_plot, ncols=3, title='Marker Gene Expression')

## 3. Generate Synthetic CAR Sequence

Next, we'll create a synthetic CAR sequence. A typical CAR construct consists of:
- Single-chain variable fragment (scFv) targeting a specific antigen
- Hinge/spacer region
- Transmembrane domain
- Co-stimulatory domains (e.g., CD28, 4-1BB)
- CD3ζ signaling domain

We'll create a simplified version of a CD19-targeting CAR.

In [ ]:
# Function to generate a random DNA sequence of specified length
def random_dna_seq(length):
    return ''.join(random.choice('ACGT') for _ in range(length))

# Generate parts of the CAR construct
signal_peptide = random_dna_seq(60)  # Signal peptide for proper membrane targeting
scfv_vh = random_dna_seq(350)        # Variable heavy chain region of antibody
linker = random_dna_seq(45)          # Linker between VH and VL
scfv_vl = random_dna_seq(320)        # Variable light chain region of antibody
hinge = random_dna_seq(150)          # Hinge/spacer region
tm_domain = random_dna_seq(120)      # Transmembrane domain
cd28_domain = random_dna_seq(180)    # CD28 co-stimulatory domain
cd3z_domain = random_dna_seq(336)    # CD3ζ signaling domain

# Combine to create full CAR sequence
car_dna_sequence = signal_peptide + scfv_vh + linker + scfv_vl + hinge + tm_domain + cd28_domain + cd3z_domain

# Create a unique gene name for the CAR
car_gene_name = "CAR_CD19"

# Display details
print(f"Generated CAR sequence: {car_gene_name}")
print(f"Length: {len(car_dna_sequence)} bp")
print(f"First 60 bp: {car_dna_sequence[:60]}...")

# Save CAR sequence to a FASTA file
car_fasta_file = "car_sequence.fa"
car_record = SeqRecord(
    Seq(car_dna_sequence),
    id=car_gene_name,
    description="Synthetic CD19-targeting CAR sequence"
)
with open(car_fasta_file, "w") as fasta_out:
    SeqIO.write(car_record, fasta_out, "fasta")

print(f"CAR sequence saved to: {car_fasta_file}")

## 4. Insert CAR Gene into the Expression Matrix

Now we'll randomly select a subset of cells to express the CAR gene. In a real CAR-T dataset, typically a fraction of cells would express the CAR construct at varying levels.

In [ ]:
# Parameters for CAR insertion
car_positive_ratio = 0.25  # Fraction of cells that will express the CAR
car_expression_mean = 3.0  # Mean expression level for CAR-positive cells
car_expression_std = 1.2   # Standard deviation of CAR expression

# Randomly select cells to express the CAR
n_car_positive = int(n_cells * car_positive_ratio)
car_positive_indices = np.random.choice(n_cells, n_car_positive, replace=False)

# Create CAR expression vector (all zeros initially)
car_expression = np.zeros(n_cells)

# Generate expression values for CAR-positive cells using a truncated normal distribution
# (truncated at 0 to ensure non-negative values)
car_values = np.random.normal(car_expression_mean, car_expression_std, n_car_positive)
car_values = np.maximum(car_values, 0)  # Ensure no negative values
car_expression[car_positive_indices] = car_values

# Add the CAR gene to the AnnData object
adata.var_names = adata.var_names.append(pd.Index([car_gene_name]))
car_expression_sparse = sparse.csr_matrix(car_expression.reshape(-1, 1))
adata_with_car = sc.AnnData(
    sparse.hstack([adata.X, car_expression_sparse]),
    obs=adata.obs,
    var=pd.DataFrame(index=adata.var_names)
)

# Update the AnnData object
adata = adata_with_car

# Add CAR expression status to cell metadata
adata.obs['car_positive'] = False
adata.obs.loc[car_positive_indices, 'car_positive'] = True
adata.obs['car_counts'] = car_expression

# Calculate updated QC metrics
sc.pp.calculate_qc_metrics(adata, inplace=True)

# Display summary
print(f"Updated expression matrix shape: {adata.shape}")
print(f"CAR-positive cells: {n_car_positive} ({car_positive_ratio:.0%})")
print(f"Mean CAR expression in positive cells: {np.mean(car_values):.2f}")

### 4.1 Visualize CAR Expression

Let's check how the CAR expression is distributed in our synthetic dataset.

In [ ]:
# Re-run PCA and UMAP with updated data
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pp.pca(adata, n_comps=50, use_highly_variable=True)
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=20)
sc.tl.umap(adata)

# Plot UMAP with CAR expression
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sc.pl.umap(adata, color='car_positive', title='CAR-positive cells', show=False)

plt.subplot(1, 2, 2)
sc.pl.umap(adata, color=car_gene_name, title='CAR expression level', show=False)

plt.tight_layout()
plt.show()

# Plot distribution of CAR expression
plt.figure(figsize=(10, 6))
plt.hist(car_expression[car_expression > 0], bins=30, alpha=0.7)
plt.title('Distribution of CAR Expression in CAR-positive Cells')
plt.xlabel('Expression Count')
plt.ylabel('Number of Cells')
plt.grid(alpha=0.3)
plt.show()

# Examine CAR expression by cell type
car_by_celltype = pd.DataFrame({
    'cell_type': adata.obs['cell_type'],
    'car_positive': adata.obs['car_positive'],
    'car_expression': adata.obs['car_counts']
})

# Calculate percentage of CAR-positive cells by cell type
car_positive_by_type = car_by_celltype.groupby('cell_type')['car_positive'].mean() * 100

plt.figure(figsize=(10, 6))
sns.barplot(x=car_positive_by_type.index, y=car_positive_by_type.values)
plt.title('Percentage of CAR-positive Cells by Cell Type')
plt.xlabel('Cell Type')
plt.ylabel('Percent CAR-positive')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 5. Save the Synthetic Dataset

Now we'll save our synthetic dataset in formats that can be used with standard scRNAseq analysis tools.

In [ ]:
# Create output directory
output_dir = Path("./synthetic_data")
output_dir.mkdir(exist_ok=True)

# Save AnnData object (compatible with scanpy)
adata_file = output_dir / "synthetic_cart_data.h5ad"
adata.write(adata_file)
print(f"Saved AnnData to: {adata_file}")

# Save as 10X-compatible format
mtx_dir = output_dir / "matrix_10x"
mtx_dir.mkdir(exist_ok=True)

# Save matrix in sparse format
mtx_file = mtx_dir / "matrix.mtx"
with open(mtx_file, 'wb') as f:
    sparse.save_npz(f, sparse.csr_matrix(adata.X))

# Save gene names (features)
features_file = mtx_dir / "features.tsv"
with open(features_file, 'w') as f:
    for gene in adata.var_names:
        f.write(f"{gene}\n")

# Save cell barcodes
barcodes_file = mtx_dir / "barcodes.tsv"
with open(barcodes_file, 'w') as f:
    for barcode in adata.obs_names:
        f.write(f"{barcode}\n")

print(f"Saved 10X-compatible files to: {mtx_dir}")

# Save cell metadata as CSV
metadata_file = output_dir / "cell_metadata.csv"
adata.obs.to_csv(metadata_file)
print(f"Saved cell metadata to: {metadata_file}")

## 6. Generate FASTQ Files for Cell Ranger Analysis (Optional)

To use this data with Cell Ranger, we would need to generate synthetic FASTQ files. This is a complex process that goes beyond the scope of this notebook, as it requires simulating reads, UMIs, and barcodes in the correct format. 

However, you can use the 10X-compatible matrix files directly for downstream analysis with scanpy, Seurat, or other scRNAseq analysis tools.

## 7. Summary and Next Steps

In this notebook, we have:

1. Created a synthetic single-cell RNA sequencing dataset with realistic T cell populations
2. Generated a synthetic CAR sequence
3. Added the CAR sequence to a random subset of cells
4. Visualized the distribution of CAR expression across cell types
5. Saved the data in formats that can be used with standard analysis tools

This synthetic dataset can be used to:
- Test and develop analysis pipelines for CAR-T scRNAseq data
- Explore methods for detecting CAR-positive cells
- Benchmark the performance of different tools without requiring real sequencing data

For a more realistic simulation that generates raw sequencing reads, consider using specialized tools like:
- [SPsimSeq](https://github.com/bvieth/spsimseq) for simulating scRNAseq counts
- [Splatter](https://bioconductor.org/packages/release/bioc/html/splatter.html) for simulating single-cell RNA sequencing data
- [polyester](https://github.com/alyssafrazee/polyester) for simulating RNA-seq reads

These can generate more realistic data that mimics technical artifacts and biases present in real scRNAseq experiments.